In [ ]:
import spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 891.9 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp

In [ ]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("ibm/MedMentions-ZS")

# Set the format to pandas to work with it easily
ds.set_format(type="pandas")

import spacy
from spacy.training import Example
from spacy.tokens import Doc, Span
from spacy import displacy
import matplotlib.pyplot as plt

# Enable GPU if available
# spacy.require_gpu()

# Initialize a blank SpaCy model
nlp = spacy.blank("en")

# Add the NER pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Function to convert tokens and ner_tags to TRAIN_DATA format
def convert_to_train_data(tokens, ner_tags):
    text = " ".join(tokens)
    entities = []
    start_idx = 0

    for token, tag in zip(tokens, ner_tags):
        token_length = len(token)

        if tag.startswith("B-"):  # Beginning of a new entity
            entity_start = start_idx
            entity_end = entity_start + token_length
            entity_label = tag[2:]
            entities.append((entity_start, entity_end, entity_label))

        elif tag.startswith("I-") and entities:  # Continuation of the current entity
            entity_end = start_idx + token_length
            # Update the last entity's end index
            entities[-1] = (entities[-1][0], entity_end, entities[-1][2])

        start_idx += token_length + 1

    return (text, {"entities": entities})

# Prepare training data from MedQA format
TRAIN_DATA = [convert_to_train_data(row.tokens, row.ner_tags) for _, row in ds['train'][:].iterrows()]
VAL_DATA = [convert_to_train_data(row.tokens, row.ner_tags) for _, row in ds['validation'][:].iterrows()]

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

from spacy.util import filter_spans

for training_example  in tqdm(TRAIN_DATA):
    # print(training_example[1])
    # print(training_example[1]["entities"])
    text = training_example[0]
    labels = training_example[1]['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 26770/26770 [00:08<00:00, 3150.42it/s]


In [ ]:
doc_bin = DocBin()

from spacy.util import filter_spans

for training_example  in tqdm(VAL_DATA):
    # print(training_example[1])
    # print(training_example[1]["entities"])
    text = training_example[0]
    labels = training_example[1]['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("val.spacy")

100%|██████████| 1289/1289 [00:00<00:00, 2250.17it/s]


In [ ]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.48    0.00    0.00    0.00    0.00
  0     200         68.17   5085.03   15.15   30.56   10.07    0.15
  0     400        216.59   5560.31   21.73   41.88   14.67    0.22
  0     600        187.98   6478.01   32.81   50.08   24.40    0.33
  0     800        210.39   7486.32   34.95   54.25   25.78    0.35
  0    1000        391.25   9095.20   43.94   49.78   39.33    0.44
  0    1200        429.98  10886.62   45.99   56.14   38.94    0.46
  0    1400        524.90  12488.96   51.64   58.73   46.07    0.52
  0    1600        662.51  14818.

In [ ]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner("")

spacy.displacy.render(doc, style="ent", jupyter=True)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
